In [1]:
!pip install selenium
!pip install selenium_stealth
!pip install selenium webdriver-manager


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 14.6 MB/s eta 0:00:00


In [2]:
# import libraries
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import pandas as pd
import numpy as np
import re
import time
import random
import os

In [3]:
# Initialize Selenium WebDriver with options
def initialize_driver_options():
    # create a ChromeOptions object
    options = Options()

    #run in headless mode
    options.add_argument("--headless")

    # disable the AutomationControlled feature of Blink rendering engine
    options.add_argument('--disable-blink-features=AutomationControlled')

    # disable pop-up blocking
    #options.add_argument('--disable-popup-blocking')
    # enable pop-up blocking
    options.add_experimental_option('excludeSwitches', ['disable-popup-blocking'])

    # start the browser window in maximized mode
    options.add_argument('--start-maximized')

    # disable extensions
    options.add_argument('--disable-extensions')

    # disable sandbox mode
    options.add_argument('--no-sandbox')

    # disable shared memory usage
    options.add_argument('--disable-dev-shm-usage')


    return options

In [4]:

# user agents list
user_agent_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                  ]
# initialise driver options
options = initialize_driver_options()

# request for a random and different user agent everytime
user_agent = random.choice(user_agent_list)

options.add_argument(f'user-agent={user_agent}')

#initialise the fortified scraper
driver = webdriver.Chrome(options=options)

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

#enable stealth mode
stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
      )


In [5]:
# get desired category links
def get_cat_links(site):
  driver.get(site)
  time.sleep(2)
  links_list = []
  #fetch the navigation links in the bbc igbo site
  for el in driver.find_elements(By.XPATH, '/html/body/header/nav/div/div/div[2]/ul'):
    for link in el.find_elements(By.TAG_NAME, 'a'):
      links = link.get_attribute('href')
      links_list.append(links)

  return links_list


In [6]:
links = get_cat_links('https://www.igboradio.com/')

In [7]:
links

['https://www.igboradio.com/',
 'https://www.igboradio.com/category/3/mmemme',
 'https://www.igboradio.com/category/5/mmepe-obodo',
 'https://www.igboradio.com/category/15/okpukperechi',
 'https://www.igboradio.com/category/7/omenaala',
 'https://www.igboradio.com/category/4/agum-akwukwo',
 'https://www.igboradio.com/category/6/edemede',
 'https://www.igboradio.com/category/16/ahu-ike',
 'https://www.igboradio.com/category/8/nchekwa',
 'https://www.igboradio.com/category/14/agha']

In [8]:
#link = get_cat_links('https://www.igboradio.com/')
for i in link:
  if i not in links:
    links.append(i)
links

NameError: name 'link' is not defined

In [9]:
links = links[1:]

In [10]:
# function to scrape each category
def scrape(site):

  # dictionary to store scraped data
  dic = {"text":[], "category": [], "source": []}

  driver.get(site)
  time.sleep(2)
  link_list = []

  # get category name
  category = site.split("/")[-1]

  # get category articles links
  for i in range(1,3):
    for el in driver.find_elements(By.CLASS_NAME, f"loop-list-{i}"):
      for link in el.find_elements(By.TAG_NAME, 'a'):
          pages = link.get_attribute('href')
          if "detail" in pages and pages not in link_list:
            link_list.append(pages)
  # extract data from articles in the page
  for i in link_list:
      driver.get(i)
      time.sleep(2)
      article = ""
      for el in driver.find_elements(By.TAG_NAME, "main"):
          article += el.text
          article += "\n"
      dic["text"].append(article)
      dic["category"].append(category)
      dic["source"].append(i)

  print("Extraction of news content from "+ category +" category successfull!")
  driver.quit()
  return dic


In [11]:
data = pd.DataFrame(columns = ["text", "category", "source"])
data

,text,category,source


In [13]:
dic = scrape(links[1])

Extraction of news content from mmepe-obodo category successfull!


In [22]:
df = pd.DataFrame(dic)
df.head()
#data = pd.concat([data, df])
#data.info()
word = df["text"].str.split()
print(word[0])

['AKUKO', 'NA-EWU', 'Anambra', 'State', 'Library', 'Board', 'Flags', 'Off', 'Summer', 'Programs', 'Aug.', '3,', '2021,', '2:09', 'p.m.', '5796', 'Views', 'Akuko', 'ifo:', '"Egbe', 'na', 'mbe"', 'Jan.', '18,', '2013,', '6:29', 'p.m.', '4073', 'Views', 'Akwa', 'ariri', 'n’onwu', 'Professor', 'Albert', 'Chinualamogu', 'Achebe', 'April', '2,', '2013,', '7:38', 'p.m.', '1838', 'Views', 'ALL', 'SECTIONS', 'Ahu-Ike', 'Okpukperechi', 'Agha', 'Oru-Ubi', 'Asusu', 'Ifo', 'Abu', 'Njem', 'Nchekwa', 'Omenaala', 'Edemede', 'Mmepe', 'Obodo', 'Agum-Akwukwo', 'Mmemme', 'Ochichi', 'Akuko', 'ANAMBRA', 'E', 'Nyela', 'Ndị', 'Ajadụ', 'Na', 'Ụfọdụ', 'Ndị', 'Nne', 'Ọzụzụ', 'Maka', 'Ịkwụrụ', 'Onwe', 'Ha', 'Izunna', 'Okafor', 'June', '27,', '2022,', '3:58', 'p.m.', '993', 'Views', 'A', 'kpọkuola', 'ụmụnwaanyị', 'ajadụ', 'na', 'ụmụnwaanyị', 'ndị', 'mụrụ', 'nwa', 'na-alụghị', 'di', 'ka', 'ha', 'guzosie', 'ike', 'ma', 'gbasikwa', 'mbọ', 'ike', "n'onwe", 'ha', 'iji', 'bụrụ', 'ndị', 'kwụụrụ', 'onwe', 'ha', 'ma', 'na-

In [ ]:
data.to_csv("igbo_radio_data.csv", index=False)

In [ ]:
new_df = pd.read_csv("igbo_news_data.csv", index_col=False)
new_dic = pd.concat([new_df, data])
new_dic.info()
new_dic.to_csv("all_igbo_news_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 833 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      833 non-null    object
 1   category  833 non-null    object
 2   source    833 non-null    object
dtypes: object(3)
memory usage: 26.0+ KB
